In [187]:
import cv2 
import numpy as np 
import colorsys
import random
import os 

def random_color():  
    h = random.uniform(0, 1)
    hsv = [h, 0.9, 1]
    color = colorsys.hsv_to_rgb(*hsv)
    color = [int(x * 255) for x in color]
    return color

def write_yolo_label_txt(filename, yolo_bboxes):
    with open(filename, 'w') as f:
        for bbox in yolo_bboxes:
            str_bbox = list(map(str, bbox))
            str_ = ' '.join(str_bbox) + '\n'
            f.write(str_)

def write_random_text_images(text_list, class_name='class_image', image_size=(448,448), num_images=10, max_objects=10, use_debug=False):
    
    rows, cols = image_size

    fontFace=cv2.FONT_HERSHEY_SIMPLEX 

    text_shuffle_list = text_list.copy()

    os.makedirs(class_name, exist_ok=True)

    for i in range(num_images):
        img = np.zeros((rows, cols, 3), dtype='uint8')
        yolo_bboxes = []
        # randint => [low, high)
        num_objects = np.random.randint(1, max_objects + 1)

        for _ in range(num_objects):
            # shuffle 
            random.shuffle(text_shuffle_list)
            # random text color and size
            text = text_shuffle_list[0]
            fontScale = np.random.randint(8,25) / 10.  
            thickness = np.random.randint(2,5)
            color = random_color() 
            #
            text_size, base_line =cv2.getTextSize(text, fontFace,fontScale,thickness)
            text_width, text_height = text_size
            org_x = np.random.randint(0, cols - text_width)
            org_y = np.random.randint(text_height, rows-text_height)
            # yolo bboxes info (class, x, y, width, height)
            class_id = text_list.index(text)
            yolo_bbox = [class_id, org_x + text_width/2.0, org_y + base_line - (text_height + base_line)/2.0, text_width, text_height + base_line]
            yolo_bboxes.append(yolo_bbox)
            # draw text 
            cv2.putText(img,text,(org_x, org_y),fontFace,fontScale,color,thickness)

        img_filename = os.path.join(class_name, f'{class_name}-{i:04d}.png')
        txt_filename = os.path.join(class_name, f'{class_name}-{i:04d}.txt')
        cv2.imwrite(img_filename, img)
        write_yolo_label_txt(txt_filename, yolo_bboxes)

        if use_debug:
            for bbox in yolo_bboxes:
                class_id, x, y, w, h = bbox
                pt1 = (int(x - w/2), int(y - h/2))
                pt2 = (int(x + w/2), int(y + h/2))
                cv2.rectangle(img, pt1, pt2, (0,200,100), 1)
            cv2.imshow('image', img)
            cv2.waitKey(0)
            cv2.destroyAllWindows()

text_list = ['dog', 'cat', 'duck']

write_random_text_images(text_list, 'dog_cat_duck', image_size=(448,448), num_images=1000, max_objects=10, use_debug=False)

